# Processamento de Linguagem Natural

Processamento inteligente de texto puro é algo muito difícil: muitas palavras são raras, palavras completamente distintas podem ter significados iguais enquanto a mesma palavra pode ter um significado completamente diferente dependendo do contexto.
O objetivo do spaCy: pegar textos puros e retornar um objeto Doc, que contém diversas informações.

Esse notebook foi inspirado nos códigos encontrados em: <http://leportella.com/pt-br/2017/11/30/brincando-de-nlp-com-spacy.html> e <https://www.analyticsvidhya.com/blog/2017/04/natural-language-processing-made-easy-using-spacy-%E2%80%8Bin-python/>

## Instalação

### Usando conda
$ conda install -c conda-forge spacy

### Usando pip
$ pip install spacy

$ python -m spacy download en

$ python -m spacy download pt

In [1]:
import spacy

## Usando um modelo pré-pronto

Podemos usar direto o modelo de português que acabamos de baixar. Para isso, separamos o modelo e chamamos ele genericamente de nlp

In [2]:
nlp = spacy.load('pt')

A partir de agora podemos usar esse modelo para entender frases em português. Vamos usar o nlp para estudar uma frase simples:

In [4]:
doc = nlp('Você encontrou o livro que eu te falei, Carla?')

In [5]:
doc

Você encontrou o livro que eu te falei, Carla?

Um Doc, objeto como aquele que acabamos de criar, é uma sequência de objetos do tipo Token e possui diversas informações sobre o texto que ele contém. Por dividir a frase em tokens, esse documento é uma estrutura iterável e portanto, deve ser acessada como tal. Já um Token é uma parte da estrutura e pode ser uma frase, palavra, uma pontuação, um espaço em branco, etc.

In [6]:
import pandas as pd
token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in doc]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: u'Yes' if x else u''))
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,Você,-20.0,,,,,Yes
1,encontrou,-20.0,,,,,Yes
2,o,-20.0,,,,,Yes
3,livro,-20.0,,,,,Yes
4,que,-20.0,Yes,,,,Yes
5,eu,-20.0,Yes,,,,Yes
6,te,-20.0,Yes,,,,Yes
7,falei,-20.0,,,,,Yes
8,",",-20.0,,Yes,,,Yes
9,Carla,-20.0,,,,,Yes


In [4]:
doc.text.split()

['Você', 'encontrou', 'o', 'livro', 'que', 'eu', 'te', 'falei,', 'Carla?']

In [5]:
[token for token in doc]

[Você, encontrou, o, livro, que, eu, te, falei, ,, Carla, ?]

In [6]:
[token.orth_ for token in doc]

['Você',
 'encontrou',
 'o',
 'livro',
 'que',
 'eu',
 'te',
 'falei',
 ',',
 'Carla',
 '?']

In [7]:
[token.orth_ for token in doc if not token.is_punct]

['Você', 'encontrou', 'o', 'livro', 'que', 'eu', 'te', 'falei', 'Carla']

In [8]:
tokens = [token for token in doc]

In [9]:
tokens[0].similarity(tokens[5])

0.29921815

In [10]:
tokens[0].similarity(tokens[3])

-0.067587666

In [11]:
[(token.orth_, token.pos_) for token in doc]

[('Você', 'PRON'),
 ('encontrou', 'VERB'),
 ('o', 'DET'),
 ('livro', 'NOUN'),
 ('que', 'PRON'),
 ('eu', 'PRON'),
 ('te', 'PRON'),
 ('falei', 'VERB'),
 (',', 'PUNCT'),
 ('Carla', 'PROPN'),
 ('?', 'PUNCT')]

In [12]:
[token.lemma_ for token in doc if token.pos_ == 'VERB']

['encontrar', 'falar']

In [13]:
doc = nlp('encontrar encontrei')
tokens = [token for token in doc]
tokens[0].is_ancestor(tokens[1])

True

In [8]:
doc = nlp('Machado de Assis um dos melhores escritores do Brasil, foi o primeiro presidente da Academia Brasileira de Letras')
doc.ents

(Machado de Assis, Brasil, Academia Brasileira de Letras)

In [9]:
[(entity, entity.label_) for entity in doc.ents]

[(Machado de Assis, 'PER'),
 (Brasil, 'LOC'),
 (Academia Brasileira de Letras, 'ORG')]

In [16]:
wiki_obama = """Barack Obama is an American politician who served as
     the 44th President of the United States from 2009 to 2017. He is the first
     African American to have served as president,
     as well as the first born outside the contiguous United States."""
nlp = spacy.load('en')
nlp_obama = nlp(wiki_obama)
[(i, i.label_) for i in nlp_obama.ents]

[(Barack Obama, 'PERSON'), (American, 'NORP'), (
       , 'PERSON'), (44th, 'ORDINAL'), (the United States,
  'GPE'), (2009 to 2017, 'DATE'), (first, 'ORDINAL'), (
       African American, 'NORP'), (
       , 'PERSON'), (first, 'ORDINAL'), (United States, 'GPE')]

Utilizando spacy para um problema de classificação
link para o dataset: <http://www.ppgia.pucpr.br/~paraiso/mineracaodeemocoes/formulario.php?download=Base_ENIAC.txt>

In [11]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

import string
punctuations = string.punctuation
parser = spacy.load('pt')

stopwords = nltk.corpus.stopwords.words('portuguese')

In [12]:
#Transformador customizado utilizando o spaCy 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Função de utilidade básica para preprocessamento do texto
def clean_text(text):     
    return text.strip().lower()


In [13]:
#Cria tokenizador spacy que pega uma sentença e cria tokens
#Este can also be replaced by word vectors 
def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [tok.lemma_.lower().strip() if tok.pos_ != "PRON" else tok.lower_ for tok in tokens]
    tokens = [tok for tok in tokens if (tok not in stopwords and tok not in punctuations)]     
    return tokens

vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
classifier = LinearSVC()
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function spacy_tokenizer at 0x121c8ba60>,
        vocabulary=None)

In [20]:
# Create the  pipeline to clean, tokenize, vectorize, and classify 
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

# Load sample data
import pandas as pd

In [21]:
data = pd.read_csv('base_eniac2.txt', names=['noticia', 'sentimento'],header=None, sep='\t')

In [22]:
data.head()

,noticia,sentimento
0,Hospital de campanha começa a funcionar em Cax...,alegria
1,Estudo indica que chimpanzés são capazes de ap...,alegria
2,STF nega ação que pedia a revogação de artigos...,alegria
3,Exército chinês faz demonstração de habilidade...,alegria
4,Zoológico alemão apresenta tigresas quadrigême...,alegria


In [23]:
data.shape

(1750, 2)

In [24]:
data.groupby('sentimento').size()

sentimento
Surpresa      1
alegria     250
desgosto    250
medo        250
neutro      250
raiva       250
surpresa    249
tristeza    250
dtype: int64

In [25]:
data['sentimento'] = data.sentimento.str.lower()

In [26]:
data.groupby('sentimento').size()

sentimento
alegria     250
desgosto    250
medo        250
neutro      250
raiva       250
surpresa    250
tristeza    250
dtype: int64

In [27]:
noticias = data.values[:,0]
labels = data.values[:,1]

In [28]:
kfold = StratifiedKFold(n_splits=10)

In [29]:
cross_val_score(estimator=pipe,X=noticias, y=labels, cv=kfold.split(noticias,labels)).mean()

0.6302857142857142